In [ ]:
import subprocess
import os
import re

In [ ]:
def burn_pgs_subtitles_into_video(source_file, output_filepath):
    
    
    # Command to burn subtitles, re-encode video, and exclude audio
    ffmpeg_command = [
        'ffmpeg',
        '-i', source_file,
        '-filter_complex', '[0:v][0:s:0]overlay',  # Overlay PGS subtitles (assuming first subtitle track)
        '-preset', 'veryfast',  # Use veryslow preset for encoding speed
        '-an',                  # Exclude audio
        output_filepath
    ]
    
    try:
        # Execute the FFmpeg command
        print(f"Processing {source_file}...")
        subprocess.run(ffmpeg_command, check=True)
        print(f"Subtitled file created: {output_filepath}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {source_file}: {e}")



In [ ]:
def process(source_dir, destination_dir):
    # Define the range of seasons (e.g., Season 1 to Season 7)
    seasons = sorted([f"Season {i}" for i in range(1, 8)])

    # Loop through each season folder
    for i, season in enumerate(seasons):
        season_path = os.path.join(source_dir, season)

        # Check if the season directory exists
        if not os.path.isdir(season_path):
            print(f"Directory {season_path} does not exist, skipping.")
            continue

        # Create corresponding destination directory for the season
        destination_season_path = os.path.join(destination_dir, season)
        os.makedirs(destination_season_path, exist_ok=True)

        # Get sorted list of files in the season folder
        files = sorted(os.listdir(season_path))

        # Process each file in the season folder
        for j, filename in enumerate(files):
            # Look for complete .mkv files
            if filename.endswith('.mkv') and not filename.endswith('.mkv.part'):

                pattern = r"S\d{2}E\d{2} - [^()]+"

                match = re.search(pattern, filename)
                if match:
                    modified_filename = match.group(0)

                video_file = os.path.join(season_path, filename)
                sup_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.sup")
                csv_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.csv")
                xml_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.xml")
                burned_videofile = os.path.join(destination_season_path, modified_filename.strip(), "video.mkv")
                if os.path.exists(burned_videofile): continue
                burn_pgs_subtitles_into_video(video_file,burned_videofile)


In [ ]:
# Specify the source and destination directories
source_directory = "/media/coof/standardHDD/Veep"  # Replace with the path to your directory containing seasons
destination_directory = "/media/coof/standardHDD/Veep2"

# Call the function to burn subtitles
process(source_directory, destination_directory)
